In [1]:
import pandas as pd 
import numpy as np

In [2]:
paths = '/Users/wyaming/Downloads/chromedownload/data_format1'
dty = {'user_id':'int32', 'item_id':'int32', 'cat_id':'int32', }
data = pd.read_csv(f'{paths}/user_log_format1.csv', dtype={'time_stamp':'str'}, nrows=10000)
data1 = pd.read_csv(f'{paths}/user_info_format1.csv', nrows=10000)
data2 = pd.read_csv(f'{paths}/train_format1.csv', nrows=10000)
data3 = pd.read_csv(f'{paths}/test_format1.csv', nrows=10000)
data_train = pd.read_csv('/Users/wyaming/Downloads/chromedownload/data_format2/train_format2.csv', nrows=100000)

In [3]:
data2['origin'] = 'train'
data3['origin'] = 'test'
matrix = pd.concat([data2, data3], ignore_index=True, sort=False)
matrix.drop(['prob'], axis=1, inplace=True)
matrix = matrix.merge(data1, on='user_id', how='left')
data.rename(columns={'seller_id':'merchant_id'}, inplace=True)

In [4]:
data['user_id'] = data['user_id'].astype('int32')
data['merchant_id'] = data['merchant_id'].astype('int32')
data['item_id'] = data['item_id'].astype('int32')
data['cat_id'] = data['cat_id'].astype('int32')
data['brand_id'].fillna(0, inplace=True)
data['brand_id'] = data['brand_id'].astype('int32')
data['time_stamp'] = pd.to_datetime(data['time_stamp'], format='%H%M')

matrix['age_range'].fillna(0, inplace=True)
matrix['gender'].fillna(2, inplace=True)
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
user_id        10000 non-null int32
item_id        10000 non-null int32
cat_id         10000 non-null int32
merchant_id    10000 non-null int32
brand_id       10000 non-null int32
time_stamp     10000 non-null datetime64[ns]
action_type    10000 non-null int64
dtypes: datetime64[ns](1), int32(5), int64(1)
memory usage: 351.7 KB


In [6]:
#特征处理
groups = data.groupby(['user_id'])
temp = groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')


In [7]:
groups = data.groupby(['merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={
    'user_id':'m2',
    'item_id':'m3', 
    'cat_id':'m4', 
    'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

In [8]:
temp = data_train[data_train['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

In [9]:
groups = data.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={
    'item_id':'um2',
    'cat_id':'um3',
    'brand_id':'um4'
})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={
    0:'um5',
    1:'um6',
    2:'um7',
    3:'um8'
})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('frist', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['frist']).dt.seconds/3600
temp.drop(['frist', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')


In [10]:
matrix['r1'] = matrix['u9']/matrix['u7']
matrix['r2'] = matrix['m8']/matrix['m6']
matrix['r3'] = matrix['um7']/matrix['um5']


In [11]:
matrix.fillna(0, inplace=True)

In [12]:
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)

In [13]:
# pd.get_dummies(matrix[['age_range','gender']])#, prefix=['age', 'gdr'])
matrix.head()

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,age_0,age_2,age_3,age_4,age_5,age_6,age_7,g_0,g_1,g_2
0,34176,3906,0.0,train,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,1
1,34176,121,0.0,train,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,1
2,34176,4356,1.0,train,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,1
3,34176,2217,0.0,train,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,1
4,230784,4818,0.0,train,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,1


In [14]:
#train-test setdata
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']


In [16]:
#导入分析库
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report


In [17]:
X_train, X_vali, y_train, y_vali = train_test_split(train_X, train_y, test_size=.3)

In [18]:
rf_clf = RandomForestClassifier(
    oob_score=True, 
    n_jobs=-1, 
    n_estimators=1000, 
    max_depth=10, 
    max_features='sqrt')

In [ ]:
rf_clf.fit(X_train, y_train)

In [22]:
print(classification_report(y_vali, rf_clf.predict(X_vali)))

              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      2785
         1.0       0.00      0.00      0.00       215

    accuracy                           0.93      3000
   macro avg       0.46      0.50      0.48      3000
weighted avg       0.86      0.93      0.89      3000



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [23]:
import xgboost as xgb 

In [24]:
model = xgb.XGBClassifier(max_depth=3, n_estimators=1000, learning_rate=0.1)

In [25]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [26]:
print(classification_report(y_vali, model.predict(X_vali)))

              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      2785
         1.0       0.33      0.00      0.01       215

    accuracy                           0.93      3000
   macro avg       0.63      0.50      0.49      3000
weighted avg       0.89      0.93      0.89      3000

